## The challenge:

In this competition you’ll predict what types of trees there are in an area based on various geographic features.

The competition datasets comes from a study conducted in four wilderness areas within the beautiful Roosevelt National Forest of northern Colorado. These areas represent forests with very little human disturbances – the existing forest cover types there are more a result of ecological processes rather than forest management practices.

The data is in raw form and contains categorical data such as wilderness areas and soil type.

## Import Packages

In [1]:
DATA_DIR = '/kaggle/input/learn-together'
DATA_DIR = 'data'

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, \
    ReduceLROnPlateau, TensorBoard

for dirname, _, filenames in os.walk(DATA_DIR):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/home/ivan/pybr/kaggle-forest-types/venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ivan/pybr/kaggle-forest-types/venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ivan/pybr/kaggle-forest-types/venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ivan/pybr/kaggle-fores

data/train.csv
data/test.csv
data/sample_submission.csv
data/sample_submission.csv.zip
data/input
data/test.csv.zip
data/train.csv.zip


/home/ivan/pybr/kaggle-forest-types/venv/lib/python3.5/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ivan/pybr/kaggle-forest-types/venv/lib/python3.5/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ivan/pybr/kaggle-forest-types/venv/lib/python3.5/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/i

In [3]:
def report(y_true, y_pred):
    print('Accuracy: %s' % accuracy_score(y_true, y_pred))
    print(classification_report(y_true, y_pred))
    print(confusion_matrix(y_true, y_pred))

## Load Dataset

In [4]:
train_df=pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
test_df=pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))

In [5]:
train_df.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,1,2596,51,3,258,0,510,221,232,148,...,0,0,0,0,0,0,0,0,0,5
1,2,2590,56,2,212,-6,390,220,235,151,...,0,0,0,0,0,0,0,0,0,5
2,3,2804,139,9,268,65,3180,234,238,135,...,0,0,0,0,0,0,0,0,0,2
3,4,2785,155,18,242,118,3090,238,238,122,...,0,0,0,0,0,0,0,0,0,2
4,5,2595,45,2,153,-1,391,220,234,150,...,0,0,0,0,0,0,0,0,0,5


In [6]:
test_df.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
0,15121,2680,354,14,0,0,2684,196,214,156,...,0,0,0,0,0,0,0,0,0,0
1,15122,2683,0,13,0,0,2654,201,216,152,...,0,0,0,0,0,0,0,0,0,0
2,15123,2713,16,15,0,0,2980,206,208,137,...,0,0,0,0,0,0,0,0,0,0
3,15124,2709,24,17,0,0,2950,208,201,125,...,0,0,0,0,0,0,0,0,0,0
4,15125,2706,29,19,0,0,2920,210,195,115,...,0,0,0,0,0,0,0,0,0,0


In [7]:
print("shape training csv: %s" % str(train_df.shape)) 
print("shape test csv: %s" % str(test_df.shape)) 

shape training csv: (15120, 56)
shape test csv: (565892, 55)


## Delete Ids
**Let's delete the Id column in the training set but store it for the test set before deleting**

In [8]:
train_df = train_df.drop(["Id"], axis = 1)

test_ids = test_df["Id"]
test_df = test_df.drop(["Id"], axis = 1)

In [9]:
train_df.columns

Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1',
       'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
       'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
       'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10',
       'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14',
       'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18',
       'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22',
       'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26',
       'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
       'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
       'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38',
       'Soil_Type39', 'Soil_Type40

In [14]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
ss = StandardScaler()
mm = MinMaxScaler()
numerical_df = train_df[['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points']]

In [15]:
X_ss = ss.fit_transform(numerical_df)

In [16]:
X_mm = mm.fit_transform(numerical_df)

In [20]:
positive_df = train_df[['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
        'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points']]
X_log = np.log(positive_df + 1)

In [21]:
X_log

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points
0,7.862112,3.951244,1.386294,5.556828,6.236370,5.402677,5.451038,5.003946,8.745125
1,7.859799,4.043051,1.098612,5.361292,5.968708,5.398163,5.463832,5.023881,8.736489
2,7.939159,4.941642,2.302585,5.594711,8.064951,5.459586,5.476464,4.912655,8.719644
3,7.932362,5.049856,2.944439,5.493061,8.036250,5.476464,5.476464,4.812184,8.734238
4,7.861727,3.828641,1.098612,5.036953,5.971262,5.398163,5.459586,5.017280,8.727940
5,7.855545,4.890349,1.945910,5.707110,4.219508,5.442418,5.472271,4.948760,8.704834
6,7.865955,3.828641,2.079442,5.602119,6.452049,5.407172,5.420535,4.934474,8.741456
7,7.865572,3.912023,1.609438,5.459586,6.352629,5.407172,5.442418,4.976734,8.736971
8,7.870166,3.828641,2.302585,5.484797,6.502790,5.411646,5.402677,4.897840,8.739536
9,7.868254,4.094345,2.397895,5.513429,6.456770,5.433722,5.393628,4.828314,8.737292


In [22]:
def categorical_features(df):
    size = len(df)
    columns_to_keep = []
    for column in df.columns:
        if 'Soil' in column or 'Wilderness' in column:
            frequency = df[column].sum() / size
            if frequency >= 0.03:
                columns_to_keep.append(column)
    print('Columns keeped %s' % columns_to_keep)
    return np.array(df[columns_to_keep])

In [23]:
categorical = categorical_features(train_df)

Columns keeped ['Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type6', 'Soil_Type10', 'Soil_Type13', 'Soil_Type17', 'Soil_Type23', 'Soil_Type29', 'Soil_Type30', 'Soil_Type32', 'Soil_Type33', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40']


In [24]:
categorical[:10]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]])

In [34]:
X = np.hstack((X_mm, categorical))

In [35]:
from sklearn.preprocessing import LabelBinarizer
binarizer = LabelBinarizer()
y = binarizer.fit_transform(train_df['Cover_Type'])

In [36]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

In [37]:
def model1():
    model = tf.keras.Sequential()
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(7, activation='softmax'))
    return model


In [38]:
def model2():
    model = tf.keras.Sequential()
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(7, activation='softmax'))
    return model


In [39]:
def model3():
    model = tf.keras.Sequential()
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(7, activation='softmax'))
    return model


In [43]:
def model4():
    model = tf.keras.Sequential()
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(7, activation='softmax'))
    return model


In [46]:
def compile_and_train(model, epochs=20, verbose=0):
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=epochs, verbose=verbose)
    print(model.metrics_names)
    print(model.evaluate(X_train, y_train))
    print(model.evaluate(X_val, y_val))

In [47]:
for model in [model1(), model2(), model3(), model4()]:
    compile_and_train(model)

['loss', 'acc']
13608/13608 [==============================] - 0s 32us/sample - loss: 0.6381 - acc: 0.7277
[0.6381080016572639, 0.7277337]
1512/1512 [==============================] - 0s 33us/sample - loss: 0.6691 - acc: 0.7216
[0.669115904147032, 0.72156084]
['loss', 'acc']
13608/13608 [==============================] - 0s 32us/sample - loss: 0.6555 - acc: 0.7166
[0.6554830388277156, 0.71663725]
1512/1512 [==============================] - 0s 27us/sample - loss: 0.6748 - acc: 0.7163
[0.6747634987982493, 0.71626985]
['loss', 'acc']
13608/13608 [==============================] - 0s 34us/sample - loss: 0.7665 - acc: 0.6828
[0.7664906509759355, 0.6827601]
1512/1512 [==============================] - 0s 30us/sample - loss: 0.7899 - acc: 0.6700
[0.7899074806738152, 0.66997355]
['loss', 'acc']
13608/13608 [==============================] - 0s 35us/sample - loss: 0.6144 - acc: 0.7404
[0.6143859773085861, 0.7404468]
1512/1512 [==============================] - 0s 36us/sample - loss: 0.6492 - a

In [48]:
model = model2()
compile_and_train(model, epochs=100, verbose=2)

Epoch 1/100
13608/13608 - 1s - loss: 1.4687 - acc: 0.3954
Epoch 2/100
13608/13608 - 1s - loss: 1.1062 - acc: 0.5318
Epoch 3/100
13608/13608 - 1s - loss: 1.0383 - acc: 0.5624
Epoch 4/100
13608/13608 - 1s - loss: 0.9904 - acc: 0.5750
Epoch 5/100
13608/13608 - 1s - loss: 0.9524 - acc: 0.5943
Epoch 6/100
13608/13608 - 1s - loss: 0.9195 - acc: 0.6082
Epoch 7/100
13608/13608 - 1s - loss: 0.8987 - acc: 0.6241
Epoch 8/100
13608/13608 - 1s - loss: 0.8838 - acc: 0.6315
Epoch 9/100
13608/13608 - 1s - loss: 0.8672 - acc: 0.6319
Epoch 10/100
13608/13608 - 1s - loss: 0.8529 - acc: 0.6400
Epoch 11/100
13608/13608 - 1s - loss: 0.8414 - acc: 0.6453
Epoch 12/100
13608/13608 - 1s - loss: 0.8266 - acc: 0.6501
Epoch 13/100
13608/13608 - 1s - loss: 0.8201 - acc: 0.6531
Epoch 14/100
13608/13608 - 1s - loss: 0.8100 - acc: 0.6587
Epoch 15/100
13608/13608 - 1s - loss: 0.8071 - acc: 0.6577
Epoch 16/100
13608/13608 - 1s - loss: 0.7998 - acc: 0.6590
Epoch 17/100
13608/13608 - 1s - loss: 0.7872 - acc: 0.6641
Epoch 

In [49]:
model = model1()
compile_and_train(model, epochs=100, verbose=2)

Epoch 1/100
13608/13608 - 1s - loss: 1.5134 - acc: 0.3667
Epoch 2/100
13608/13608 - 1s - loss: 1.1232 - acc: 0.5189
Epoch 3/100
13608/13608 - 1s - loss: 1.0351 - acc: 0.5543
Epoch 4/100
13608/13608 - 1s - loss: 0.9782 - acc: 0.5813
Epoch 5/100
13608/13608 - 1s - loss: 0.9333 - acc: 0.6046
Epoch 6/100
13608/13608 - 1s - loss: 0.9010 - acc: 0.6195
Epoch 7/100
13608/13608 - 1s - loss: 0.8785 - acc: 0.6348
Epoch 8/100
13608/13608 - 1s - loss: 0.8592 - acc: 0.6399
Epoch 9/100
13608/13608 - 1s - loss: 0.8447 - acc: 0.6477
Epoch 10/100
13608/13608 - 1s - loss: 0.8335 - acc: 0.6499
Epoch 11/100
13608/13608 - 1s - loss: 0.8237 - acc: 0.6555
Epoch 12/100
13608/13608 - 1s - loss: 0.8056 - acc: 0.6617
Epoch 13/100
13608/13608 - 1s - loss: 0.8029 - acc: 0.6688
Epoch 14/100
13608/13608 - 1s - loss: 0.7928 - acc: 0.6708
Epoch 15/100
13608/13608 - 1s - loss: 0.7875 - acc: 0.6725
Epoch 16/100
13608/13608 - 1s - loss: 0.7839 - acc: 0.6722
Epoch 17/100
13608/13608 - 1s - loss: 0.7776 - acc: 0.6749
Epoch 

In [50]:
model = model4()
compile_and_train(model, epochs=100, verbose=2)

Epoch 1/100
13608/13608 - 1s - loss: 1.3445 - acc: 0.4322
Epoch 2/100
13608/13608 - 1s - loss: 1.0219 - acc: 0.5661
Epoch 3/100
13608/13608 - 1s - loss: 0.9473 - acc: 0.5945
Epoch 4/100
13608/13608 - 1s - loss: 0.8910 - acc: 0.6191
Epoch 5/100
13608/13608 - 1s - loss: 0.8547 - acc: 0.6368
Epoch 6/100
13608/13608 - 1s - loss: 0.8273 - acc: 0.6531
Epoch 7/100
13608/13608 - 1s - loss: 0.8013 - acc: 0.6567
Epoch 8/100
13608/13608 - 1s - loss: 0.7902 - acc: 0.6598
Epoch 9/100
13608/13608 - 1s - loss: 0.7818 - acc: 0.6702
Epoch 10/100
13608/13608 - 1s - loss: 0.7676 - acc: 0.6745
Epoch 11/100
13608/13608 - 1s - loss: 0.7520 - acc: 0.6802
Epoch 12/100
13608/13608 - 1s - loss: 0.7429 - acc: 0.6834
Epoch 13/100
13608/13608 - 1s - loss: 0.7344 - acc: 0.6882
Epoch 14/100
13608/13608 - 1s - loss: 0.7256 - acc: 0.6936
Epoch 15/100
13608/13608 - 1s - loss: 0.7215 - acc: 0.6966
Epoch 16/100
13608/13608 - 1s - loss: 0.7143 - acc: 0.6964
Epoch 17/100
13608/13608 - 1s - loss: 0.7042 - acc: 0.7032
Epoch 

## Final training

### Callbacks

In [51]:
tensorboard_logs = TensorBoard(log_dir='./logs', histogram_freq=1,
                               write_graph=True, write_images=True,
                               update_freq='epoch')
mcp_save = ModelCheckpoint(os.path.join('./keras_models', 
                                        'modelweights.{epoch:02d}-{val_loss:.2f}.hdf5'),
                           save_best_only=True, monitor='val_acc', mode='min')
early_stop = EarlyStopping(monitor='val_loss', patience=50, verbose=0, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=20,
                              verbose=1, min_delta=1e-2, mode='min')

### Model

In [52]:
def model_final():
    model = tf.keras.Sequential()
    model.add(layers.Dense(128, activation='relu',
              kernel_regularizer=tf.keras.regularizers.l2(0.0001)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(64, activation='relu',
              kernel_regularizer=tf.keras.regularizers.l2(0.0002)))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu',
              kernel_regularizer=tf.keras.regularizers.l2(0.0002)))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(7, activation='softmax'))
    return model



In [53]:
def model_final2():
    model = tf.keras.Sequential()
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(7, activation='softmax'))
    return model


### Training

In [54]:
model = model_final2()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=500,
          callbacks=[early_stop, reduce_lr, mcp_save, tensorboard_logs],
          validation_data=(X_val, y_val))


Train on 13608 samples, validate on 1512 samples
Epoch 1/500
13608/13608 [==============================] - 2s 161us/sample - loss: 1.2937 - acc: 0.4611 - val_loss: 0.9556 - val_acc: 0.5880
Epoch 2/500
13608/13608 [==============================] - 1s 69us/sample - loss: 1.0095 - acc: 0.5608 - val_loss: 0.8985 - val_acc: 0.6310
Epoch 3/500
13608/13608 [==============================] - 1s 69us/sample - loss: 0.9423 - acc: 0.5928 - val_loss: 0.8357 - val_acc: 0.6488
Epoch 4/500
13608/13608 [==============================] - 1s 69us/sample - loss: 0.8870 - acc: 0.6242 - val_loss: 0.7894 - val_acc: 0.6647
Epoch 5/500
13608/13608 [==============================] - 1s 68us/sample - loss: 0.8597 - acc: 0.6365 - val_loss: 0.7633 - val_acc: 0.6759
Epoch 6/500
13608/13608 [==============================] - 1s 69us/sample - loss: 0.8301 - acc: 0.6471 - val_loss: 0.7366 - val_acc: 0.6984
Epoch 7/500
13608/13608 [==============================] - 1s 69us/sample - loss: 0.8095 - acc: 0.6566 - val_l

13608/13608 [==============================] - 1s 67us/sample - loss: 0.6082 - acc: 0.7474 - val_loss: 0.5584 - val_acc: 0.7765
Epoch 58/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.6147 - acc: 0.7453 - val_loss: 0.5461 - val_acc: 0.7844
Epoch 59/500
13608/13608 [==============================] - 1s 68us/sample - loss: 0.6165 - acc: 0.7410 - val_loss: 0.5519 - val_acc: 0.7824
Epoch 60/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.6115 - acc: 0.7449 - val_loss: 0.5528 - val_acc: 0.7765
Epoch 61/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.6109 - acc: 0.7454 - val_loss: 0.5513 - val_acc: 0.7751
Epoch 62/500
13608/13608 [==============================] - 1s 68us/sample - loss: 0.6070 - acc: 0.7446 - val_loss: 0.5485 - val_acc: 0.7837
Epoch 63/500
13608/13608 [==============================] - 1s 68us/sample - loss: 0.6044 - acc: 0.7541 - val_loss: 0.5347 - val_acc: 0.7837
Epoch 64/500
13608/13608 [

Epoch 114/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.5699 - acc: 0.7660 - val_loss: 0.5131 - val_acc: 0.7996
Epoch 115/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.5697 - acc: 0.7659 - val_loss: 0.5092 - val_acc: 0.7989
Epoch 116/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.5676 - acc: 0.7654 - val_loss: 0.5210 - val_acc: 0.7923
Epoch 117/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.5636 - acc: 0.7647 - val_loss: 0.5101 - val_acc: 0.7956
Epoch 118/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.5609 - acc: 0.7659 - val_loss: 0.5051 - val_acc: 0.8029
Epoch 119/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.5603 - acc: 0.7714 - val_loss: 0.5184 - val_acc: 0.7910
Epoch 120/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.5643 - acc: 0.7655 - val_loss: 0.5077 - val_acc: 0.7897
Epoch 

13608/13608 [==============================] - 1s 67us/sample - loss: 0.5349 - acc: 0.7803 - val_loss: 0.4919 - val_acc: 0.8049
Epoch 170/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.5366 - acc: 0.7788 - val_loss: 0.4911 - val_acc: 0.8062
Epoch 171/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.5307 - acc: 0.7835 - val_loss: 0.4845 - val_acc: 0.8075
Epoch 172/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.5337 - acc: 0.7834 - val_loss: 0.4935 - val_acc: 0.8056
Epoch 173/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.5323 - acc: 0.7828 - val_loss: 0.4910 - val_acc: 0.8115
Epoch 174/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.5246 - acc: 0.7786 - val_loss: 0.4951 - val_acc: 0.8095
Epoch 175/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.5348 - acc: 0.7794 - val_loss: 0.4959 - val_acc: 0.8062
Epoch 176/500
13608/

13608/13608 [==============================] - 1s 67us/sample - loss: 0.5136 - acc: 0.7913 - val_loss: 0.4805 - val_acc: 0.8168
Epoch 225/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.5157 - acc: 0.7851 - val_loss: 0.4854 - val_acc: 0.8155
Epoch 226/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.5134 - acc: 0.7906 - val_loss: 0.4903 - val_acc: 0.8168
Epoch 227/500
13608/13608 [==============================] - 1s 68us/sample - loss: 0.5136 - acc: 0.7905 - val_loss: 0.4805 - val_acc: 0.8161
Epoch 228/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.5136 - acc: 0.7925 - val_loss: 0.4851 - val_acc: 0.8108
Epoch 229/500
13608/13608 [==============================] - 1s 66us/sample - loss: 0.5107 - acc: 0.7911 - val_loss: 0.4910 - val_acc: 0.8095
Epoch 230/500
13440/13608 [============================>.] - ETA: 0s - loss: 0.5158 - acc: 0.7916
Epoch 00230: ReduceLROnPlateau reducing learning rate to 0.00026

13608/13608 [==============================] - 1s 67us/sample - loss: 0.4882 - acc: 0.7993 - val_loss: 0.4779 - val_acc: 0.8194
Epoch 279/500
13608/13608 [==============================] - 1s 68us/sample - loss: 0.5008 - acc: 0.7981 - val_loss: 0.4765 - val_acc: 0.8228
Epoch 280/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.5044 - acc: 0.7953 - val_loss: 0.4811 - val_acc: 0.8241
Epoch 281/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.5018 - acc: 0.7983 - val_loss: 0.4837 - val_acc: 0.8194
Epoch 282/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.5004 - acc: 0.7981 - val_loss: 0.4798 - val_acc: 0.8201
Epoch 283/500
13608/13608 [==============================] - 1s 68us/sample - loss: 0.4986 - acc: 0.7950 - val_loss: 0.4795 - val_acc: 0.8221
Epoch 284/500
13608/13608 [==============================] - 1s 68us/sample - loss: 0.5002 - acc: 0.7945 - val_loss: 0.4744 - val_acc: 0.8214
Epoch 285/500
13608/

13608/13608 [==============================] - 1s 66us/sample - loss: 0.4876 - acc: 0.8020 - val_loss: 0.4779 - val_acc: 0.8201
Epoch 333/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.4919 - acc: 0.7994 - val_loss: 0.4722 - val_acc: 0.8254
Epoch 334/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.4956 - acc: 0.7967 - val_loss: 0.4755 - val_acc: 0.8254
Epoch 335/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.4926 - acc: 0.7989 - val_loss: 0.4758 - val_acc: 0.8254
Epoch 336/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.4887 - acc: 0.8067 - val_loss: 0.4766 - val_acc: 0.8241
Epoch 337/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.4984 - acc: 0.7985 - val_loss: 0.4736 - val_acc: 0.8247
Epoch 338/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.4959 - acc: 0.7983 - val_loss: 0.4764 - val_acc: 0.8214
Epoch 339/500
13608/

13608/13608 [==============================] - 1s 67us/sample - loss: 0.4909 - acc: 0.7992 - val_loss: 0.4765 - val_acc: 0.8274
Epoch 388/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.4893 - acc: 0.7998 - val_loss: 0.4770 - val_acc: 0.8221
Epoch 389/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.4834 - acc: 0.8043 - val_loss: 0.4767 - val_acc: 0.8221
Epoch 390/500
13504/13608 [============================>.] - ETA: 0s - loss: 0.4941 - acc: 0.8004
Epoch 00390: ReduceLROnPlateau reducing learning rate to 4.398046876303852e-05.
13608/13608 [==============================] - 1s 67us/sample - loss: 0.4940 - acc: 0.8002 - val_loss: 0.4759 - val_acc: 0.8254
Epoch 391/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.4920 - acc: 0.8042 - val_loss: 0.4759 - val_acc: 0.8247
Epoch 392/500
13608/13608 [==============================] - 1s 67us/sample - loss: 0.4885 - acc: 0.8017 - val_loss: 0.4763 - val_acc: 0.824

In [55]:
tf.__version__

'1.14.0'

In [56]:
model1 = model_final2()
model1.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model1.fit(X_train, y_train, epochs=500)
print(model1.metrics_names)
print(model1.evaluate(X_train, y_train))
print(model1.evaluate(X_val, y_val))

Epoch 1/500
13608/13608 [==============================] - 1s 74us/sample - loss: 1.3395 - acc: 0.4448
Epoch 2/500
13608/13608 [==============================] - 1s 62us/sample - loss: 1.0194 - acc: 0.5683
Epoch 3/500
13608/13608 [==============================] - 1s 61us/sample - loss: 0.9606 - acc: 0.5949
Epoch 4/500
13608/13608 [==============================] - 1s 62us/sample - loss: 0.9012 - acc: 0.6160
Epoch 5/500
13608/13608 [==============================] - 1s 61us/sample - loss: 0.8709 - acc: 0.6296
Epoch 6/500
13608/13608 [==============================] - 1s 61us/sample - loss: 0.8394 - acc: 0.6427
Epoch 7/500
13608/13608 [==============================] - 1s 62us/sample - loss: 0.8198 - acc: 0.6503
Epoch 8/500
13608/13608 [==============================] - 1s 61us/sample - loss: 0.8022 - acc: 0.6567
Epoch 9/500
13608/13608 [==============================] - 1s 62us/sample - loss: 0.7836 - acc: 0.6669
Epoch 10/500
13608/13608 [==============================] - 1s 62us/sampl

13608/13608 [==============================] - 1s 60us/sample - loss: 0.5854 - acc: 0.7568
Epoch 80/500
13608/13608 [==============================] - 1s 60us/sample - loss: 0.5916 - acc: 0.7549
Epoch 81/500
13608/13608 [==============================] - 1s 60us/sample - loss: 0.5972 - acc: 0.7521
Epoch 82/500
13608/13608 [==============================] - 1s 60us/sample - loss: 0.5947 - acc: 0.7521
Epoch 83/500
13608/13608 [==============================] - 1s 61us/sample - loss: 0.5920 - acc: 0.7536
Epoch 84/500
13608/13608 [==============================] - 1s 61us/sample - loss: 0.5845 - acc: 0.7567
Epoch 85/500
13608/13608 [==============================] - 1s 61us/sample - loss: 0.5949 - acc: 0.7556
Epoch 86/500
13608/13608 [==============================] - 1s 61us/sample - loss: 0.5878 - acc: 0.7556
Epoch 87/500
13608/13608 [==============================] - 1s 60us/sample - loss: 0.5850 - acc: 0.7561
Epoch 88/500
13608/13608 [==============================] - 1s 61us/sample - 

13608/13608 [==============================] - 1s 60us/sample - loss: 0.5610 - acc: 0.7664
Epoch 158/500
13608/13608 [==============================] - 1s 60us/sample - loss: 0.5578 - acc: 0.7732
Epoch 159/500
13608/13608 [==============================] - 1s 60us/sample - loss: 0.5536 - acc: 0.7702
Epoch 160/500
13608/13608 [==============================] - 1s 61us/sample - loss: 0.5519 - acc: 0.7701
Epoch 161/500
13608/13608 [==============================] - 1s 60us/sample - loss: 0.5551 - acc: 0.7713
Epoch 162/500
13608/13608 [==============================] - 1s 61us/sample - loss: 0.5542 - acc: 0.7736
Epoch 163/500
13608/13608 [==============================] - 1s 60us/sample - loss: 0.5447 - acc: 0.7803
Epoch 164/500
13608/13608 [==============================] - 1s 61us/sample - loss: 0.5582 - acc: 0.7720
Epoch 165/500
13608/13608 [==============================] - 1s 61us/sample - loss: 0.5448 - acc: 0.7788
Epoch 166/500
13608/13608 [==============================] - 1s 61us/

13608/13608 [==============================] - 1s 61us/sample - loss: 0.5349 - acc: 0.7814
Epoch 236/500
13608/13608 [==============================] - 1s 60us/sample - loss: 0.5345 - acc: 0.7835
Epoch 237/500
13608/13608 [==============================] - 1s 60us/sample - loss: 0.5356 - acc: 0.7840
Epoch 238/500
13608/13608 [==============================] - 1s 60us/sample - loss: 0.5280 - acc: 0.7851
Epoch 239/500
13608/13608 [==============================] - 1s 60us/sample - loss: 0.5400 - acc: 0.7823
Epoch 240/500
13608/13608 [==============================] - 1s 61us/sample - loss: 0.5288 - acc: 0.7840
Epoch 241/500
13608/13608 [==============================] - 1s 60us/sample - loss: 0.5361 - acc: 0.7815
Epoch 242/500
13608/13608 [==============================] - 1s 60us/sample - loss: 0.5327 - acc: 0.7842
Epoch 243/500
13608/13608 [==============================] - 1s 60us/sample - loss: 0.5351 - acc: 0.7837
Epoch 244/500
13608/13608 [==============================] - 1s 60us/

13608/13608 [==============================] - 1s 60us/sample - loss: 0.5215 - acc: 0.7887
Epoch 314/500
13608/13608 [==============================] - 1s 60us/sample - loss: 0.5308 - acc: 0.7873
Epoch 315/500
13608/13608 [==============================] - 1s 61us/sample - loss: 0.5266 - acc: 0.7866
Epoch 316/500
13608/13608 [==============================] - 1s 60us/sample - loss: 0.5259 - acc: 0.7896
Epoch 317/500
13608/13608 [==============================] - 1s 61us/sample - loss: 0.5249 - acc: 0.7893
Epoch 318/500
13608/13608 [==============================] - 1s 61us/sample - loss: 0.5296 - acc: 0.7844
Epoch 319/500
13608/13608 [==============================] - 1s 61us/sample - loss: 0.5266 - acc: 0.7884
Epoch 320/500
13608/13608 [==============================] - 1s 61us/sample - loss: 0.5260 - acc: 0.7867
Epoch 321/500
13608/13608 [==============================] - 1s 61us/sample - loss: 0.5237 - acc: 0.7895
Epoch 322/500
13608/13608 [==============================] - 1s 61us/

13608/13608 [==============================] - 1s 63us/sample - loss: 0.5203 - acc: 0.7880
Epoch 392/500
13608/13608 [==============================] - 1s 63us/sample - loss: 0.5156 - acc: 0.7937
Epoch 393/500
13608/13608 [==============================] - 1s 66us/sample - loss: 0.5215 - acc: 0.7912
Epoch 394/500
13608/13608 [==============================] - 1s 62us/sample - loss: 0.5125 - acc: 0.7953
Epoch 395/500
13608/13608 [==============================] - 1s 62us/sample - loss: 0.5148 - acc: 0.7932
Epoch 396/500
13608/13608 [==============================] - 1s 65us/sample - loss: 0.5169 - acc: 0.7937
Epoch 397/500
13608/13608 [==============================] - 1s 64us/sample - loss: 0.5202 - acc: 0.7913
Epoch 398/500
13608/13608 [==============================] - 1s 69us/sample - loss: 0.5204 - acc: 0.7909
Epoch 399/500
13608/13608 [==============================] - 1s 64us/sample - loss: 0.5106 - acc: 0.7963
Epoch 400/500
13608/13608 [==============================] - 1s 64us/

13608/13608 [==============================] - 1s 59us/sample - loss: 0.5150 - acc: 0.7956
Epoch 470/500
13608/13608 [==============================] - 1s 61us/sample - loss: 0.5151 - acc: 0.7951
Epoch 471/500
13608/13608 [==============================] - 1s 61us/sample - loss: 0.5111 - acc: 0.7948
Epoch 472/500
13608/13608 [==============================] - 1s 63us/sample - loss: 0.5095 - acc: 0.7973
Epoch 473/500
13608/13608 [==============================] - 1s 60us/sample - loss: 0.5120 - acc: 0.7975
Epoch 474/500
13608/13608 [==============================] - 1s 63us/sample - loss: 0.5107 - acc: 0.7931
Epoch 475/500
13608/13608 [==============================] - 1s 66us/sample - loss: 0.5124 - acc: 0.7919
Epoch 476/500
13608/13608 [==============================] - 1s 63us/sample - loss: 0.5135 - acc: 0.7917
Epoch 477/500
13608/13608 [==============================] - 1s 61us/sample - loss: 0.5109 - acc: 0.7949
Epoch 478/500
13608/13608 [==============================] - 1s 62us/

In [57]:
y_pred = model1.predict(X_val)
report(binarizer.inverse_transform(y_val), binarizer.inverse_transform(y_pred))

Accuracy: 0.8194444444444444
              precision    recall  f1-score   support

           1       0.75      0.72      0.74       225
           2       0.72      0.60      0.65       214
           3       0.85      0.68      0.76       206
           4       0.91      0.98      0.94       209
           5       0.83      0.92      0.87       226
           6       0.74      0.85      0.79       201
           7       0.92      0.97      0.94       231

    accuracy                           0.82      1512
   macro avg       0.82      0.82      0.81      1512
weighted avg       0.82      0.82      0.82      1512

[[162  37   0   0   6   1  19]
 [ 46 129   4   0  26   8   1]
 [  0   2 141  12   7  44   0]
 [  0   0   3 204   0   2   0]
 [  0   8   4   0 208   6   0]
 [  0   4  14   9   3 171   0]
 [  7   0   0   0   0   0 224]]


In [58]:
y_pred = model.predict(X_val)
report(binarizer.inverse_transform(y_val), binarizer.inverse_transform(y_pred))

Accuracy: 0.8247354497354498
              precision    recall  f1-score   support

           1       0.75      0.74      0.75       225
           2       0.75      0.62      0.68       214
           3       0.83      0.70      0.76       206
           4       0.94      0.97      0.95       209
           5       0.84      0.92      0.88       226
           6       0.73      0.85      0.79       201
           7       0.92      0.95      0.94       231

    accuracy                           0.82      1512
   macro avg       0.82      0.82      0.82      1512
weighted avg       0.82      0.82      0.82      1512

[[167  34   0   0   6   0  18]
 [ 44 132   4   0  28   6   0]
 [  0   2 145   7   3  49   0]
 [  0   0   4 203   0   2   0]
 [  1   7   4   0 209   5   0]
 [  0   2  18   7   3 171   0]
 [ 11   0   0   0   0   0 220]]


## Predictions

In [49]:
test_pred = model.predict(test_df)

In [51]:
# Save test predictions to file
output = pd.DataFrame({'ID': test_ids,
                       'Cover_Type': test_pred})
output.to_csv('submission.csv', index=False)